## 잉여현금흐름 모형(FCFM) 기업가치

In [ ]:
# # 현금흐름할인모형(DCF)  = 향후 일정기간 FCF 현재가치 합 + 일정기간 후 잔여가치 의 현재가치
# # 향후 5개년도 매출액 예상
# 만약 2019년 매출액이 15조원이라면

In [1]:
import numpy as np
import pandas as pd
years=['2019A', '2020F', '2021F', '2022F', '2023F', '2024F']
sales = pd.Series(index=years)
sales['2019A'] = 15
sales

C:\Users\JYB\AppData\Local\Temp/ipykernel_11988/428913726.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sales = pd.Series(index=years)


2019A    15.0
2020F     NaN
2021F     NaN
2022F     NaN
2023F     NaN
2024F     NaN
dtype: float64

In [ ]:
# 매출액 : 성장율을 고려하여 열의 나머지 부분을 추정매출 수치로 채우는 것
# 또한 간단한 for 루프 를 사용 하여 나머지 판매 값을 계산하는 프로세스를 반복
# 2020 년 매출은 2019 년에 (1+ 성장율 %)를 곱한 값
# 성장율은 10% 상정 : 해당기업의 ROE 고려 (유보율 X ROE)  , 요구수익률 = 배당수익률 +  성장률
# 미래 연도 매출은 전년도 매출에 (1+ 판매 성장 %)를 곱한 값

In [14]:
growth_rate = 0.1
# Loop to populate the data series of sales
for year in range(1,6):
    sales[year] = sales[year-1] * (1+growth_rate)

In [ ]:
# 세후순영업이익(NOPAT) 계산

In [3]:

ebitda_margin = 0.20  #  EBITDA 마진 20 %를 사용하여 EBITDA를 계산
depr_percent = 0.03   #     감가상각비율 동기업 추이 반영 3% 
ebitda = sales * ebitda_margin   # ebitda = 매출액 X 마진
depreciation = sales * depr_percent #     감가상각비 및 상각비를 차감. 
ebit = ebitda - depreciation  
tax_rate = 0.30     # 실효법인세율  30%  동기업 반영
tax_payment = -ebit * tax_rate
tax_payment = tax_payment.apply(lambda x: min(x,0))
nopat = ebit + tax_payment
nopat  # 세후순영업이익

#     그 후 EBIT가 양수일 때만 세금 비용을 계산하는 함수를 사용하여 세금 비용을 계산
   

2019A    1.785000
2020F    1.963500
2021F    2.159850
2022F    2.375835
2023F    2.613419
2024F    2.874760
dtype: float64

In [ ]:
#  순운전자본 (NWC) 계산. (회사의 유동 자산 -  현재 부채)의 차이
# 증가는 미래에 지불해야하는 자산을 받았음을 의미하기 때문에 순운전자본의 변화만 관심 
# 순운전자본이 1 년에서 다음 해로 증가하면 현금 감소 
# 자본적지출(capex) : 판매 비율 감안 계산

In [6]:
nwc_percent = 0.24
nwc = sales * nwc_percent
change_in_nwc = nwc.shift(1) - nwc
capex_percent = depr_percent
capex = -(sales * capex_percent)

In [7]:
# FCF  계산
free_cash_flow = nopat + depreciation + capex + change_in_nwc
free_cash_flow

2019A         NaN
2020F    1.603500
2021F    1.763850
2022F    1.940235
2023F    2.134259
2024F    2.347684
dtype: float64

In [ ]:
# 5 년의 FCF가 예측 된 5 년 DCF 와 잔여가치 고려
# 잔여가치 성장율(terminal_growth) 고려   2%
# WACC 고려하여  기업가치 산정  :  WACC = 10%  

In [8]:
cost_of_capital = 0.10
terminal_growth = 0.02
terminal_value = ((free_cash_flow[-1] * (1 + terminal_growth))/
                 (cost_of_capital - terminal_growth))
discount_factors = [(1 / (1 + cost_of_capital)) ** i for i in range (1,6)]
dcf_value = (sum(free_cash_flow[1:]*discount_factors) +
            terminal_value * discount_factors[-1])
dcf_value

25.874659090909088

In [ ]:
#  행 항목을 행으로, 연도를 열로 원하므로 행 항목을 색인으로 설정. 
# 또한 모든 값을 소수점 이하 2 자리로 반올림하여 일치

In [11]:
# Exporting the Data to Excel
output = pd.DataFrame([sales, ebit, tax_payment, nopat, 
                       depreciation, capex, change_in_nwc,
                       free_cash_flow],
                     index=["Sales", "EBIT", "Tax Expense", 
                            "NOPAT", "D&A Expense",
                            "Capital Expenditures",
                            "Increase in NWC",
                            "Free Cash Flow"]).round(2)
output.to_excel('Python DCF Model.xlsx')
output

,2019A,2020F,2021F,2022F,2023F,2024F
Sales,15.00,16.50,18.15,19.97,21.96,24.16
EBIT,2.55,2.80,3.09,3.39,3.73,4.11
Tax Expense,-0.76,-0.84,-0.93,-1.02,-1.12,-1.23
NOPAT,1.78,1.96,2.16,2.38,2.61,2.87
D&A Expense,0.45,0.50,0.54,0.60,0.66,0.72
Capital Expenditures,-0.45,-0.50,-0.54,-0.60,-0.66,-0.72
Increase in NWC,NaN,-0.36,-0.40,-0.44,-0.48,-0.53
Free Cash Flow,NaN,1.60,1.76,1.94,2.13,2.35


In [10]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

def load_FS():
    print("자료의 소스는 FnGuide, 네이버금융, Yahoo Finance입니다.해당 사이트들에 필요한 정보가 누락되어있으면 valuation을 중단합니다.\n")
    company = input("종목코드를 입력하세요: ")
    source_url = '''https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'''.format(company)
    tables = pd.read_html(source_url, encoding='utf-8')
    
    IS = tables[0]
    BS = tables[2]
    CF = tables[4]
    
    return IS, BS, CF, company
   
    
    
class step1:
    def __init__(self, company, IS, BS, CF):
        self._company = company
        self._IS = IS
        self._BS = BS
        self._CF = CF
        self._tax = 0
        self._sales = [] 
        self._cogs = []
        self._sgna = []
        self._op = []
        self._capex_l = []
        self._dep_l = []
        self._amo_l = []
        self._dNWC = [0]
        self._FCFF = []
        self._NWC = []
        
    def Basic_IS(self):

        IS_core = self._IS.loc[[0,1,3],['IFRS(연결)','2018/12','2019/12','2020/12']]
        self._sales = list(IS_core.loc[0])
        self._cogs = list(IS_core.loc[1])
        self._sgna = list(IS_core.loc[3])
        self._sales.pop(0)
        self._cogs.pop(0)
        self._sgna.pop(0)
        
        
        for i in range(3):
            if self._sales[i] > 3000:
                self._tax = 0.275
            elif 200 < self._sales[i] <= 3000:
                self._tax = 0.242
            elif 2 < self._sales[i] <= 200:
                self._tax = 0.22
            else :
                self._tax = 0.11
                
            self._op.append((self._sales[i]-self._cogs[i]-self._sgna[i])*(1-self._tax))
        
        return self._tax
    
    def Basic_CF(self): ##sales 


        url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'.format(self._company)
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            capex = soup.select_one('#divCashY > table > tbody > tr:nth-child(108)')
            dep = soup.select_one('#divCashY > table > tbody > tr:nth-child(9)')
            amo =  soup.select_one('#divCashY > table > tbody > tr:nth-child(10)')
        else : 
            print(response.status_code)


        capex = capex.get_text()
        dep = dep.get_text()
        amo = amo.get_text()

        n = re.compile('\d+\,?\d+')
        self._capex_l = n.findall(capex)
        self._dep_l = n.findall(dep)
        self._amo_l = n.findall(amo)

        for i in range(4):
            self._capex_l[i] = int(self._capex_l[i].replace(',',''))
            self._dep_l[i] = int(self._dep_l[i].replace(',',''))
            self._amo_l[i] = int(self._amo_l[i].replace(',',''))

        self._capex_l.pop()
        self._dep_l.pop()
        self._amo_l.pop()




    def Dnwc(self):
        url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'.format(self._company)
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            AR_r = soup.select_one('#divDaechaY > table > tbody > tr:nth-child(6)').get_text()
            Inventory_r =  soup.select_one('#divDaechaY > table > tbody > tr:nth-child(3)').get_text()
            CA_etc_r =  soup.select_one('#divDaechaY > table > tbody > tr:nth-child(11)').get_text()
            AP_r = soup.select_one('#divDaechaY > table > tbody > tr:nth-child(35)').get_text()
            CL_etc_r = soup.select_one('#divDaechaY > table > tbody > tr:nth-child(42)').get_text()
        else : 
            print(response.status_code)

        ft = re.compile('\d+\,?\d+')

        AR = ft.findall(AR_r)
        Inventory = ft.findall(Inventory_r)
        CA_etc = ft.findall(CA_etc_r)
        AP = ft.findall(AP_r)
        CL_etc = ft.findall(CL_etc_r)

        for i in range(len(AR)):
            AR[i] = int(AR[i].replace(',',''))
            Inventory[i] = int(Inventory[i].replace(',',''))
            CA_etc[i] = int(CA_etc[i].replace(',',''))
            AP[i] = int(AP[i].replace(',',''))
            CL_etc[i] = int(CL_etc[i].replace(',',''))
            self._NWC.append((AR[i]+Inventory[i]+CA_etc[i]) - (AP[i]+CL_etc[i]))

        self._NWC.pop()

        for i in range(len(self._op)):
            if i < 2:
                self._dNWC.append(self._NWC[i+1]-self._NWC[i])


    def FCFF(self):
    
        for i in range(3):
            self._FCFF.append(self._op[i]+self._dep_l[i]+self._amo_l[i]-self._capex_l[i]-self._dNWC[i])

    def mktable(self):
        d_main_info = {'매출액': self._sales,'매출원가': self._cogs,'판관비': self._sgna
                 ,'영업이익': self._op,'+유형자산상각비':self._dep_l,'+무형자산상각비': self._amo_l,
                   '-Capex': self._capex_l,'-dNwc': self._dNWC, 'FCFF': self._FCFF, '참고용NWC': self._NWC}
        
        yr1 = (datetime.datetime.today() - relativedelta(years = 1)).strftime("%Y")
        yr2 = (datetime.datetime.today() - relativedelta(years = 2)).strftime("%Y")
        yr3 = (datetime.datetime.today() - relativedelta(years = 3)).strftime("%Y")
        Main_info = pd.DataFrame(d_main_info,index=[yr3, yr2 ,yr1]) 


        S_BS = Main_info.transpose()
        print("최근 3개년 FCFF(단위: 억)")
        display(S_BS)
        print("첫 번째 열의 dNWC의 경우 해당 페이지에서 얻을 수 없으며, 미래 추정에 영향을 주지 않음\n ")
        return S_BS